# Bank Churn Prediction

### Objective
Given a Bank customer, build a neural network based classifier that can determine whether they will leave or not in the next 6 months.

### Context
Businesses like banks which provide service have to worry about problem of 'Churn' i.e. customers leaving and joining another service provider. It is important to understand which aspects of the service influence a customer's decision in this regard. Management can concentrate efforts on improvement of service, keeping in mind these priorities.

### Data Decsritpion
The dataset contains 10,000 sample points with 14 distinct features such as CustomerId, CreditScore, Geography, Gender, Age, Tenure, Balance etc.

### Data Dictonary
1. RowNumber: Row number.
2. CustomerId: Unique identification key for different customers.
3. Surname: Surname of the customer
4. Credit Score: Credit score is a measure of an individual's ability to pay back the borrowed amount. It is the numerical representation of their creditworthiness. A credit score is a 3-digit number that falls in the range of 300-900, 900 being the highest.
5. Geography: The country to which the customer belongs.
6. Gender: The gender of the customer.
7. Age: Age of the customer.
8. Tenure: The period of time a customer has been associated with the bank.
9. Balance: The account balance (the amount of money deposited in the bank account) of the customer.
10. NumOfProducts: How many accounts, bank account affiliated products the person has.
11. HasCrCard: Does the customer have a credit card through the bank?
12. IsActiveMember: Subjective, but for the concept
13. EstimatedSalary: Estimated salary of the customer.
14. Exited: Did they leave the bank after all?

#### Read Data Set

In [40]:
import tensorflow as tf

import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, auc
import matplotlib.pyplot as plt
%matplotlib inline 

from tensorflow.keras import optimizers

In [3]:
tf.__version__

'2.1.0'

In [4]:
bank_data = pd.read_csv('bank.csv')
bank_data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# Check for missing value
bank_data.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [6]:
# Cheking for any null values in the data set.
bank_data.isnull().values.any()

False

In [7]:
# Since there is no null values so not removing any null from the data set.

#### Drop the columns which are unique for all users like IDs (5 points)

In [8]:
# In the bank dataset since RowNumber and CustomerId are unique identfications for the customer hence dropping those columns.

In [9]:
bank_data = bank_data.drop("RowNumber", axis = 1)

In [10]:
bank_data = bank_data.drop("CustomerId", axis = 1)

In [11]:
bank_data

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [12]:
# Check for duplicate data
dups = bank_data.duplicated()
print('Number of duplicate rows = %d' % (dups.sum()))

bank_data[dups]

Number of duplicate rows = 0


,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited


In [13]:
# Dropping any duplicate data.
bank_data.drop_duplicates()

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [14]:
# Since the rows and column are same after dropping duplicates if any looks like there are no duplicates in the bank dataset.

In [15]:
# This gives the no. of unique values for each column. 
bank_data.nunique()

Surname            2932
CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

#### Distinguish the feature and target set (5 points)

In [16]:
# Performing descriptive analysis.
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Surname          10000 non-null  object 
 1   CreditScore      10000 non-null  int64  
 2   Geography        10000 non-null  object 
 3   Gender           10000 non-null  object 
 4   Age              10000 non-null  int64  
 5   Tenure           10000 non-null  int64  
 6   Balance          10000 non-null  float64
 7   NumOfProducts    10000 non-null  int64  
 8   HasCrCard        10000 non-null  int64  
 9   IsActiveMember   10000 non-null  int64  
 10  EstimatedSalary  10000 non-null  float64
 11  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 937.6+ KB


In [17]:
# Creating dummy variables for the categorical features.
bank_data = pd.get_dummies(bank_data)

# Converting the bank dataset to float in order to be able to convert it into tensors later.
bank_data = bank_data.astype('float32') 

# We want to see all the column names 
pd.set_option('display.max_columns', None)
bank_data

CreditScore   Age  Tenure        Balance  NumOfProducts  HasCrCard  \
0           619.0  42.0     2.0       0.000000            1.0        1.0   
1           608.0  41.0     1.0   83807.859375            1.0        0.0   
2           502.0  42.0     8.0  159660.796875            3.0        1.0   
3           699.0  39.0     1.0       0.000000            2.0        0.0   
4           850.0  43.0     2.0  125510.820312            1.0        1.0   
...           ...   ...     ...            ...            ...        ...   
9995        771.0  39.0     5.0       0.000000            2.0        1.0   
9996        516.0  35.0    10.0   57369.609375            1.0        1.0   
9997        709.0  36.0     7.0       0.000000            1.0        0.0   
9998        772.0  42.0     3.0   75075.312500            2.0        1.0   
9999        792.0  28.0     4.0  130142.789062            1.0        1.0   

      IsActiveMember  EstimatedSalary  Exited  Surname_Abazu  Surname_Abbie  \
0                1.0    101348.882812     1.0            0.0            0.0   
1                1.0    112542.578125     0.0            0.0            0.0   
2                0.0    113931.570312     1.0            0.0            0.0   
3                0.0     93826.632812     0.0            0.0            0.0   
4                1.0     79084.101562     0.0            0.0            0.0   
...              ...              ...     ...            ...            ...   
9995             0.0     96270.640625     0.0            0.0            0.0   
9996             1.0    101699.773438     0.0            0.0            0.0   
9997             1.0     42085.578125     1.0            0.0            0.0   
9998             0.0     92888.523438     1.0            0.0            0.0   
9999             0.0     38190.781250     0.0            0.0            0.0   

      Surname_Abbott  Surname_Abdullah  Surname_Abdulov  Surname_Abel  \
0                0.0               0.0              0.0           0.0   
1                0.0               0.0              0.0           0.0   
2                0.0               0.0              0.0           0.0   
3                0.0               0.0              0.0           0.0   
4                0.0               0.0              0.0           0.0   
...              ...               ...              ...           ...   
9995             0.0               0.0              0.0           0.0   
9996             0.0               0.0              0.0           0.0   
9997             0.0               0.0              0.0           0.0   
9998             0.0               0.0              0.0           0.0   
9999             0.0               0.0              0.0           0.0   

      Surname_Abernathy  Surname_Abramov  Surname_Abramova  \
0                   0.0              0.0               0.0   
1                   0.0              0.0               0.0   
2                   0.0              0.0               0.0   
3                   0.0              0.0               0.0   
4                   0.0              0.0               0.0   
...                 ...              ...               ...   
9995                0.0              0.0               0.0   
9996                0.0              0.0               0.0   
9997                0.0              0.0               0.0   
9998                0.0              0.0               0.0   
9999                0.0              0.0               0.0   

      Surname_Abramovich  Surname_Abramowitz  Surname_Abrego  Surname_Abron  \
0                    0.0                 0.0             0.0            0.0   
1                    0.0                 0.0             0.0            0.0   
2                    0.0                 0.0             0.0            0.0   
3                    0.0                 0.0             0.0            0.0   
4                    0.0                 0.0             0.0            0.0   
...                  ...                 ...    

In [18]:
# Now we are getting the features and labels and CreditScore as our target variable in the bank dataset.

In [19]:
bank_data.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Surname_Abazu',
       ...
       'Surname_Zubarev', 'Surname_Zubareva', 'Surname_Zuev', 'Surname_Zuyev',
       'Surname_Zuyeva', 'Geography_France', 'Geography_Germany',
       'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object', length=2946)

#### Divide the data set into training and test sets (5 points)

In [20]:
from sklearn.model_selection import train_test_split
X = bank_data[['Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Exited']]
Y = bank_data['CreditScore']

In [21]:
# Splitting the test and train data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

# Scaling the data using MinMaxScalar as features are on different scales
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

#### Normalize the train and test data(10points)

In [22]:
# Fitting the transform on test and train separately
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[0.14864865, 0.3       , 0.45096785, ..., 1.        , 0.623664  ,
        0.        ],
       [0.14864865, 0.4       , 0.42155474, ..., 0.        , 0.2051895 ,
        0.        ],
       [0.39189193, 0.6       , 0.51823777, ..., 1.        , 0.22843051,
        1.        ],
       ...,
       [0.36486486, 0.90000004, 0.        , ..., 1.        , 0.459958  ,
        1.        ],
       [0.28378376, 0.8       , 0.        , ..., 1.        , 0.5056917 ,
        0.        ],
       [0.32432434, 0.6       , 0.39920887, ..., 0.        , 0.16695698,
        1.        ]], dtype=float32)

In [23]:
# Converting the data elements into tensors since we need it for tensorflow based operations.

X_train=tf.convert_to_tensor(X_train)
y_train=tf.convert_to_tensor(y_train.values)
X_test=tf.convert_to_tensor(X_test)
y_test=tf.convert_to_tensor(y_test.values)

In [24]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8000, 8)
(2000, 8)
(8000,)
(2000,)


#### Initialize & build the model. Identify the points of improvement and implement the same. (20)

###### Model 1:

In [25]:
input_dim = X_train.shape[1]
output_dim = 1
learning_rate = 0.01

# Initializing the weights and bias variables. 
weights = tf.Variable(tf.zeros(shape=(input_dim, output_dim), dtype= tf.float32))
bias = tf.Variable(tf.ones(shape=(output_dim,), dtype= tf.float32))

In [26]:
# Using matmul for matrix multiplication and is used for calculating predictions
def predict(features):
  return tf.matmul(features, weights) + bias 

# Mean square error as we are doing Regression
def compute_loss(y_true, predictions):
  return tf.reduce_mean(tf.square(y_true - predictions)) 

# Defining function to train the model.
def train(x, y):
  with tf.GradientTape() as tape:
    predictions = predict(x)
    loss = compute_loss(y, predictions)
    dloss_dw, dloss_db = tape.gradient(loss, [weights, bias]) 
  weights.assign_sub(learning_rate * dloss_dw)
  bias.assign_sub(learning_rate * dloss_db)
  return loss

In [27]:
# The smaller the loss the better the model.

In [28]:
# Calling the train function with 20 epochs
for epoch in range(20):
  loss = train(X_train, y_train)
  print('Epoch %d: Loss = %.4f' % (epoch, float(loss)))


print('Final Weights after 20 epochs:')
print('')
print(weights)

print('Final Bias after 20 epochs:')
print('')
print(bias)

Epoch 0: Loss = 430953.4062
Epoch 1: Loss = 389729.2812
Epoch 2: Loss = 352675.4688
Epoch 3: Loss = 319369.7812
Epoch 4: Loss = 289432.4688
Epoch 5: Loss = 262522.4375
Epoch 6: Loss = 238333.0000
Epoch 7: Loss = 216588.6094
Epoch 8: Loss = 197041.5938
Epoch 9: Loss = 179469.3906
Epoch 10: Loss = 163671.9688
Epoch 11: Loss = 149469.5938
Epoch 12: Loss = 136700.7031
Epoch 13: Loss = 125220.1094
Epoch 14: Loss = 114897.3359
Epoch 15: Loss = 105615.1406
Epoch 16: Loss = 97268.0938
Epoch 17: Loss = 89761.5469
Epoch 18: Loss = 83010.3359
Epoch 19: Loss = 76938.0156
Final Weights after 20 epochs:

<tf.Variable 'Variable:0' shape=(8, 1) dtype=float32, numpy=
array([[ 46.786304],
       [ 82.48097 ],
       [ 50.323338],
       [ 29.537405],
       [112.91427 ],
       [ 81.21526 ],
       [ 82.18004 ],
       [ 33.08197 ]], dtype=float32)>
Final Bias after 20 epochs:

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([168.19568], dtype=float32)>


In [33]:
# From the above we got our weights and the bias is 168.19568

###### Model 2:

In [43]:
# Creating Keras model with Sequential class
model = Sequential()

In [44]:
# Keras layers can be added to the model
model.add(Dense(64, input_shape = (29,), activation = 'relu'))
model.add(Dense(32, activation = 'tanh'))

In [45]:
# Compiling Keras model before training data
sgd = optimizers.Adam(lr = 0.001)

In [46]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1920      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
Total params: 4,000
Trainable params: 4,000
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Traning the Keras Model
model.fit(X_train, y_train.values, batch_size = 700, epochs = 10, verbose = 1)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'values'

#### Predict the results using 0.5 as a threshold (10points)

###### Model 1:

In [35]:
# From the Epoch 19 we see Loss as 76938 but this is no the real loss. Hence using Mean square error so taking the square root.
76938.0156**0.5

277.37702788803546

In [36]:
# In training data the loss is predicted as 277 which is huge but not very bad.

In [37]:
# Testing model and predicitng on test data
test_predictions = tf.matmul(X_test, weights) + bias
print(compute_loss(y_test, test_predictions)**0.5)

tf.Tensor(267.9615, shape=(), dtype=float32)


In [38]:
# In testing data the loss is 268 which is better than training loss prediction.

###### Model 2:

In [53]:
# Evaluating Keras model
X_test = preprocessing.normalize(X_test)

In [54]:
results = model.evaluate(X_test, y_test.values)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'values'

In [55]:
print(model.metrics_names)
print(results)

['loss', 'accuracy']


NameError: name 'results' is not defined

#### Print the Accuracy score and confusion matrix (5points)

In [56]:
Y_pred = model.predict_classes(X_test, batch_size=200, verbose=0)
print('Accuracy Model1 (Dropout): '+ str(model.evaluate(X_test,y_test.values)[1]))
print('Recall_score: ' + str(recall_score(y_test.values,Y_pred)))
print('Precision_score: ' + str(precision_score(y_test.values, Y_pred)))
print('F-score: ' + str(f1_score(y_test.values,Y_pred)))
confusion_matrix(y_test.values, Y_pred)

ValueError: Error when checking input: expected dense_input to have shape (29,) but got array with shape (8,)